# Weather Data Streaming with Azure Event Hub and Databricks

This notebook demonstrates real-time weather data streaming using:
- Azure Event Hub for message handling
- Weather API for data collection
- Databricks for processing and streaming
- Azure Key Vault for secure credential management

## Prerequisites
This notebook requires:
- Databricks environment (uses `dbutils` for secret management)
- Configured Azure Key Vault with required secrets:
  - `eventhub-connection-string`: Connection string for Azure Event Hub
  - `weather-api-key`: API key for WeatherAPI.com
- Azure Event Hub instance named 'eventhub-weather-streaming'

## Architecture Overview
1. Weather data is fetched from WeatherAPI.com
2. Data is processed and transformed
3. Processed data is streamed to Azure Event Hub
4. PySpark streaming handles continuous data processing

### PySpark Streaming Concepts Used:
- `spark.readStream`: Creates streaming DataFrame
- `format("rate")`: Generates continuous test data
- `writeStream`: Writes streaming data
- `foreachBatch`: Custom processing for each micro-batch
- `awaitTermination`: Keeps the streaming query running

## 1. Event Hub Connection Test
Testing the Event Hub connection by sending a sample event. This validates:
- Key Vault access for secure credential management
- Event Hub connectivity and configuration
- Event sending functionality

Note: The `dbutils.secrets` utility is specific to Databricks and will only work within a Databricks environment.

In [0]:
## Testing: send a test event (in jason format) to eventhub from databricks and monitor event within the eventhub data explorer
## using secret scope to let databricks read secrets from key vault
from azure.eventhub import EventHubProducerClient, EventData
import json
import requests

# Event Hub configuration

# Getting secret value from Key Vault
eventhub_connection_string = dbutils.secrets.get(scope="key-vault-scope", key="eventhub-connection-string")

EVENT_HUB_NAME = "eventhub-weather-streaming"

# Initialize the Event Hub producer
producer = EventHubProducerClient.from_connection_string(conn_str=eventhub_connection_string, eventhub_name=EVENT_HUB_NAME)

# Function to send events to Event Hub
def send_event(event):
    event_data_batch = producer.create_batch()
    event_data_batch.add(EventData(json.dumps(event)))
    producer.send_batch(event_data_batch)

# Sample JSON event
event = {
    "event_id": 2222,
    "event_name": "Key Vault Test",
}

# Send the event
send_event(event)

# Close the producer
producer.close()

# check eventhub data explorer

## 2. Weather API Integration Test
Testing the Weather API connection and data retrieval. This cell:
- Fetches current weather data for Vancouver
- Uses secured API key from Key Vault (via Databricks utilities)
- Demonstrates the basic weather data structure

In [0]:
# Testing API: get current weather from weather api as an example
weatherapikey = dbutils.secrets.get(scope="key-vault-scope", key="weather-api-key")
location = "Vancouver" 

base_url = "http://api.weatherapi.com/v1/"

current_weather_url = f"{base_url}/current.json"

params = {
    'key': weatherapikey,
    'q': location,
}
response = requests.get(current_weather_url, params=params)

if response.status_code == 200:
    current_weather = response.json()
    print("Current Weather:")
    print(json.dumps(current_weather, indent=3))
else:
    print(f"Error: {response.status_code}, {response.text}")

## 3. Extended Weather Data Collection
Implementing comprehensive weather data collection including:
- Current weather conditions
- Air quality measurements
- Weather alerts
- 3-day forecast

The data is structured into a flattened format for easier processing and streaming.

In [0]:
# Testing: get current weather (plus air quality index), forecast weather, and alerts from weather api
import requests
import json

# Function to handle the API response
def handle_response(response):
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}, {response.text}"

# Function to get current weather and air quality data
def get_current_weather(base_url, api_key, location):
    current_weather_url = f"{base_url}/current.json"
    params = {
        'key': api_key,
        'q': location,
        "aqi": 'yes'
    }
    response = requests.get(current_weather_url, params=params)
    return handle_response(response)

# Function to get Forecast Data
def get_forecast_weather(base_url, api_key, location, days):
    forecast_url = f"{base_url}/forecast.json"
    params = {
        "key": api_key,
        "q": location,
        "days": days,
    }
    response = requests.get(forecast_url, params=params)
    return handle_response(response)

# Function to get Alerts
def get_alerts(base_url, api_key, location):
    alerts_url = f"{base_url}/alerts.json"
    params = {
        'key': api_key,
        'q': location,
        "alerts": 'yes'
    }
    response = requests.get(alerts_url, params=params)
    return handle_response(response)

# Flatten and merge the data
def flatten_data(current_weather, forecast_weather, alerts):
    location_data = current_weather.get("location", {})
    current = current_weather.get("current", {})
    condition = current.get("condition", {})
    air_quality = current.get("air_quality", {})
    forecast = forecast_weather.get("forecast", {}).get("forecastday", [])
    alert_list = alerts.get("alerts", {}).get("alert", [])

    flattened_data = {
        'name': location_data.get('name'),
        'region': location_data.get('region'),
        'country': location_data.get('country'),
        'lat': location_data.get('lat'),
        'lon': location_data.get('lon'),
        'localtime': location_data.get('localtime'),
        'temp_c': current.get('temp_c'),
        'is_day': current.get('is_day'),
        'condition_text': condition.get('text'),
        'condition_icon': condition.get('icon'),
        'wind_kph': current.get('wind_kph'),
        'wind_degree': current.get('wind_degree'),
        'wind_dir': current.get('wind_dir'),
        'pressure_in': current.get('pressure_in'),
        'precip_in': current.get('precip_in'),
        'humidity': current.get('humidity'),
        'cloud': current.get('cloud'),
        'feelslike_c': current.get('feelslike_c'),
        'uv': current.get('uv'),
        'air_quality': {
            'co': air_quality.get('co'),
            'no2': air_quality.get('no2'),
            'o3': air_quality.get('o3'),
            'so2': air_quality.get('so2'),
            'pm2_5': air_quality.get('pm2_5'),
            'pm10': air_quality.get('pm10'),
            'us-epa-index': air_quality.get('us-epa-index'),
            'gb-defra-index': air_quality.get('gb-defra-index')
        },
        'alerts': [
            {
                'headline': alert.get('headline'),
                'severity': alert.get('severity'),
                'description': alert.get('desc'),
                'instruction': alert.get('instruction')
            }
            for alert in alert_list
        ],
        'forecast': [
            {
                'date': day.get('date'),
                'maxtemp_c': day.get('day', {}).get('maxtemp_c'),
                'mintemp_c': day.get('day', {}).get('mintemp_c'),
                'condition': day.get('day', {}).get('condition', {}).get('text')
            }
            for day in forecast
        ]
    }
    return flattened_data

# Main program
def fetch_weather_data():

    base_url = "http://api.weatherapi.com/v1/"
    location = "Vancouver"  # You can replace with any city name based on your preference
    weatherapikey = dbutils.secrets.get(scope="key-vault-scope", key="weather-api-key")

    # Get data from API
    current_weather = get_current_weather(base_url, weatherapikey, location)
    forecast_weather = get_forecast_weather(base_url, weatherapikey, location, 3)
    alerts = get_alerts(base_url, weatherapikey, location)

    # Flatten and merge data
    merged_data = flatten_data(current_weather, forecast_weather, alerts)
    print("Weather Data:", json.dumps(merged_data, indent=3))

# Calling the Main Program
fetch_weather_data()

In [ ]:
''' #response
Weather Data: {
   "name": "Vancouver",
   "region": "British Columbia",
   "country": "Canada",
   "lat": 49.25,
   "lon": -123.1333,
   "localtime": "2025-03-29 00:11",
   "temp_c": 9.3,
   "is_day": 0,
   "condition_text": "Light rain",
   "condition_icon": "//cdn.weatherapi.com/weather/64x64/night/296.png",
   "wind_kph": 16.9,
   "wind_degree": 132,
   "wind_dir": "SE",
   "pressure_in": 29.89,
   "precip_in": 0.0,
   "humidity": 87,
   "cloud": 100,
   "feelslike_c": 6.8,
   "uv": 0.0,
   "air_quality": {
      "co": 392.2,
      "no2": 50.69,
      "o3": 46.0,
      "so2": 9.065,
      "pm2_5": 12.025,
      "pm10": 13.505,
      "us-epa-index": 1,
      "gb-defra-index": 2
   },
   "alerts": [],
   "forecast": [
      {
         "date": "2025-03-29",
         "maxtemp_c": 9.3,
         "mintemp_c": 6.1,
         "condition": "Patchy rain nearby"
      },
      {
         "date": "2025-03-30",
         "maxtemp_c": 13.2,
         "mintemp_c": 5.3,
         "condition": "Partly Cloudy "
      },
      {
         "date": "2025-03-31",
         "maxtemp_c": 7.3,
         "mintemp_c": 4.9,
         "condition": "Heavy rain"
      }
   ]
}
'''

## 4. Single Event Weather Data Stream
Integration of weather data collection with Event Hub streaming. This implementation:
- Combines the weather data collection functionality
- Formats data for Event Hub transmission
- Sends a single comprehensive weather data event

In [0]:
# Combining: sending the complete weather data as an event to eventhub (single event)

import requests
import json
from azure.eventhub import EventHubProducerClient, EventData

# Event Hub configuration
EVENT_HUB_NAME = "eventhub-weather-streaming"
eventhub_connection_string = dbutils.secrets.get(scope="key-vault-scope", key="eventhub-connection-string")

# Initialize the Event Hub producer
producer = EventHubProducerClient.from_connection_string(conn_str=eventhub_connection_string, eventhub_name=EVENT_HUB_NAME)

# Function to send events to Event Hub
def send_event(event):
    event_data_batch = producer.create_batch()
    event_data_batch.add(EventData(json.dumps(event)))
    producer.send_batch(event_data_batch)

# Function to handle the API response
def handle_response(response):
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}, {response.text}"

# Function to get current weather and air quality data
def get_current_weather(base_url, api_key, location):
    current_weather_url = f"{base_url}/current.json"
    params = {
        'key': api_key,
        'q': location,
        "aqi": 'yes'
    }
    response = requests.get(current_weather_url, params=params)
    return handle_response(response)

# Function to get Forecast Data
def get_forecast_weather(base_url, api_key, location, days):
    forecast_url = f"{base_url}/forecast.json"
    params = {
        "key": api_key,
        "q": location,
        "days": days,
    }
    response = requests.get(forecast_url, params=params)
    return handle_response(response)

# Function to get Alerts
def get_alerts(base_url, api_key, location):
    alerts_url = f"{base_url}/alerts.json"
    params = {
        'key': api_key,
        'q': location,
        "alerts": 'yes'
    }
    response = requests.get(alerts_url, params=params)
    return handle_response(response)

# Flatten and merge the data
def flatten_data(current_weather, forecast_weather, alerts):
    location_data = current_weather.get("location", {})
    current = current_weather.get("current", {})
    condition = current.get("condition", {})
    air_quality = current.get("air_quality", {})
    forecast = forecast_weather.get("forecast", {}).get("forecastday", [])
    alert_list = alerts.get("alerts", {}).get("alert", [])

    flattened_data = {
        'name': location_data.get('name'),
        'region': location_data.get('region'),
        'country': location_data.get('country'),
        'lat': location_data.get('lat'),
        'lon': location_data.get('lon'),
        'localtime': location_data.get('localtime'),
        'temp_c': current.get('temp_c'),
        'is_day': current.get('is_day'),
        'condition_text': condition.get('text'),
        'condition_icon': condition.get('icon'),
        'wind_kph': current.get('wind_kph'),
        'wind_degree': current.get('wind_degree'),
        'wind_dir': current.get('wind_dir'),
        'pressure_in': current.get('pressure_in'),
        'precip_in': current.get('precip_in'),
        'humidity': current.get('humidity'),
        'cloud': current.get('cloud'),
        'feelslike_c': current.get('feelslike_c'),
        'uv': current.get('uv'),
        'air_quality': {
            'co': air_quality.get('co'),
            'no2': air_quality.get('no2'),
            'o3': air_quality.get('o3'),
            'so2': air_quality.get('so2'),
            'pm2_5': air_quality.get('pm2_5'),
            'pm10': air_quality.get('pm10'),
            'us-epa-index': air_quality.get('us-epa-index'),
            'gb-defra-index': air_quality.get('gb-defra-index')
        },
        'alerts': [
            {
                'headline': alert.get('headline'),
                'severity': alert.get('severity'),
                'description': alert.get('desc'),
                'instruction': alert.get('instruction')
            }
            for alert in alert_list
        ],
        'forecast': [
            {
                'date': day.get('date'),
                'maxtemp_c': day.get('day', {}).get('maxtemp_c'),
                'mintemp_c': day.get('day', {}).get('mintemp_c'),
                'condition': day.get('day', {}).get('condition', {}).get('text')
            }
            for day in forecast
        ]
    }
    return flattened_data

# Main program
def fetch_weather_data():
    base_url = "http://api.weatherapi.com/v1/"
    location = "Vancouver"
    weatherapikey = dbutils.secrets.get(scope="key-vault-scope", key="weather-api-key")

    # Get data from API
    current_weather = get_current_weather(base_url, weatherapikey, location)
    forecast_weather = get_forecast_weather(base_url, weatherapikey, location, 3)
    alerts = get_alerts(base_url, weatherapikey, location)

    # Flatten and merge data
    merged_data = flatten_data(current_weather, forecast_weather, alerts)

    # Sending the weather data to Event HUB
    send_event(merged_data)

# Calling the Main Program
fetch_weather_data()

## 5. Continuous Weather Data Streaming
Implementation of continuous weather data streaming using PySpark Structured Streaming:

### Key Components:
- **PySpark Streaming**: Uses `spark.readStream` to create a streaming source
- **Rate Source**: Generates events at specified intervals (1 event per second)
- **Batch Processing**: Uses `foreachBatch` to process and send weather data
- **Error Handling**: Includes try-catch blocks for robust error management

### PySpark Streaming Concepts Used:
- `spark.readStream`: Creates streaming DataFrame
- `format("rate")`: Generates continuous test data
- `writeStream`: Writes streaming data
- `foreachBatch`: Custom processing for each micro-batch
- `awaitTermination`: Keeps the streaming query running

Note: The streaming application runs continuously until manually stopped.

In [0]:
# Sending the complete weather data in a streaming fashion to eventhub

from azure.eventhub import EventHubProducerClient, EventData
import json
import requests
from datetime import datetime

# Event Hub configuration
EVENT_HUB_NAME = "eventhub-weather-streaming"
eventhub_connection_string = dbutils.secrets.get(scope="key-vault-scope", key="eventhub-connection-string")
weatherapikey = dbutils.secrets.get(scope="key-vault-scope", key="weather-api-key")

# Initialize the Event Hub producer
producer = EventHubProducerClient.from_connection_string(conn_str=eventhub_connection_string, eventhub_name=EVENT_HUB_NAME)

# Function to send events to Event Hub
def send_event(event):
    event_data_batch = producer.create_batch()
    event_data_batch.add(EventData(json.dumps(event)))
    producer.send_batch(event_data_batch)

def fetch_weather_data():
    base_url = "http://api.weatherapi.com/v1/"
    location = "Vancouver"
    
    current_weather = get_current_weather(base_url, weatherapikey, location)
    forecast_weather = get_forecast_weather(base_url, weatherapikey, location, 3)
    alerts = get_alerts(base_url, weatherapikey, location)
    
    return flatten_data(current_weather, forecast_weather, alerts)

# Batch processing function for streaming
def process_batch(batch_df, batch_id):
    try:     
        # Fetch weather data
        weather_data = fetch_weather_data()
        
        # Add timestamp
        weather_data['timestamp'] = datetime.now().isoformat()
        
        # Send the weather data
        send_event(weather_data)
        
        print(f"Event Sent at {datetime.now()}")

    except Exception as e:
        print(f"Error sending events in batch {batch_id}: {str(e)}")
        raise e

# Set up a streaming source (for example, rate source for testing purposes)
streaming_df = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# Write the streaming data using foreachBatch to send weather data to Event Hub
query = streaming_df.writeStream \
    .foreachBatch(process_batch) \
    .start()

# Keep the streaming query running
query.awaitTermination()

# Close the producer after termination
producer.close()

In [ ]:
# Sending the weather data to EventHub every 30 seconds
from azure.eventhub import EventHubProducerClient, EventData
import json
import requests
from datetime import datetime, timedelta

# Event Hub configuration
EVENT_HUB_NAME = "eventhub-weather-streaming"
eventhub_connection_string = dbutils.secrets.get(
    scope="key-vault-scope", key="eventhub-connection-string"
)
weatherapikey = dbutils.secrets.get(scope="key-vault-scope", key="weather-api-key")

# Initialize the Event Hub producer
producer = EventHubProducerClient.from_connection_string(
    conn_str=eventhub_connection_string, eventhub_name=EVENT_HUB_NAME
)


# Function to send events to Event Hub
def send_event(event):
    event_data_batch = producer.create_batch()
    event_data_batch.add(EventData(json.dumps(event)))
    producer.send_batch(event_data_batch)


# Function to handle the API response
def handle_response(response):
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}, {response.text}"


# Function to get current weather and air quality data
def get_current_weather(base_url, api_key, location):
    current_weather_url = f"{base_url}/current.json"
    params = {"key": api_key, "q": location, "aqi": "yes"}
    response = requests.get(current_weather_url, params=params)
    return handle_response(response)


# Function to get Forecast Data
def get_forecast_weather(base_url, api_key, location, days):
    forecast_url = f"{base_url}/forecast.json"
    params = {
        "key": api_key,
        "q": location,
        "days": days,
    }
    response = requests.get(forecast_url, params=params)
    return handle_response(response)

# Function to get Alerts
def get_alerts(base_url, api_key, location):
    alerts_url = f"{base_url}/forecast.json"
    params = {
        'key': api_key,
        'q': location,
        "alerts": 'yes'
    }
    response = requests.get(alerts_url, params=params)
    return handle_response(response)

# Flatten and merge the data
def flatten_data(current_weather, forecast_weather, alerts):
    location_data = current_weather.get("location", {})
    current = current_weather.get("current", {})
    condition = current.get("condition", {})
    air_quality = current.get("air_quality", {})
    forecast = forecast_weather.get("forecast", {}).get("forecastday", [])
    alert_list = alerts.get("alerts", {}).get("alert", [])

    flattened_data = {
        "name": location_data.get("name"),
        "region": location_data.get("region"),
        "country": location_data.get("country"),
        "lat": location_data.get("lat"),
        "lon": location_data.get("lon"),
        "localtime": location_data.get("localtime"),
        "temp_c": current.get("temp_c"),
        "is_day": current.get("is_day"),
        "condition_text": condition.get("text"),
        "condition_icon": condition.get("icon"),
        "wind_kph": current.get("wind_kph"),
        "wind_degree": current.get("wind_degree"),
        "wind_dir": current.get("wind_dir"),
        "pressure_in": current.get("pressure_in"),
        "precip_in": current.get("precip_in"),
        "humidity": current.get("humidity"),
        "cloud": current.get("cloud"),
        "feelslike_c": current.get("feelslike_c"),
        "uv": current.get("uv"),
        "air_quality": {
            "co": air_quality.get("co"),
            "no2": air_quality.get("no2"),
            "o3": air_quality.get("o3"),
            "so2": air_quality.get("so2"),
            "pm2_5": air_quality.get("pm2_5"),
            "pm10": air_quality.get("pm10"),
            "us-epa-index": air_quality.get("us-epa-index"),
            "gb-defra-index": air_quality.get("gb-defra-index"),
        },
        "alerts": [
            {
                "headline": alert.get("headline"),
                "severity": alert.get("severity"),
                "description": alert.get("desc"),
                "instruction": alert.get("instruction"),
            }
            for alert in alert_list
        ],
        "forecast": [
            {
                "date": day.get("date"),
                "maxtemp_c": day.get("day", {}).get("maxtemp_c"),
                "mintemp_c": day.get("day", {}).get("mintemp_c"),
                "condition": day.get("day", {}).get("condition", {}).get("text"),
            }
            for day in forecast
        ],
    }
    return flattened_data


def fetch_weather_data():
    base_url = "http://api.weatherapi.com/v1/"
    location = (
        "Vancouver"  # You can replace with any city name based on your preference
    )
    weatherapikey = dbutils.secrets.get(scope="key-vault-scope", key="weather-api-key")

    # Get data from API
    current_weather = get_current_weather(base_url, weatherapikey, location)
    forecast_weather = get_forecast_weather(base_url, weatherapikey, location, 3)
    alerts = get_alerts(base_url, weatherapikey, location)

    # Flatten and merge data
    merged_data = flatten_data(current_weather, forecast_weather, alerts)
    return merged_data


# Function to process each batch of streaming data
last_sent_time = datetime.now() - timedelta(seconds=30)  # Initialize last sent time


# Main program
def process_batch(batch_df, batch_id):
    global last_sent_time
    try:
        # Get current time
        current_time = datetime.now()

        # Check if 30 seconds have passed since last event was sent
        if (current_time - last_sent_time).total_seconds() >= 30:
            # Fetch weather data
            weather_data = fetch_weather_data()

            # Send the weather data (current weather part)
            send_event(weather_data)

            # Update last sent time
            last_sent_time = current_time
            print(f"Event Sent at {last_sent_time}")

    except Exception as e:
        print(f"Error sending events in batch {batch_id}: {str(e)}")
        raise e


# Set up a streaming source (for example, rate source for testing purposes)
streaming_df = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# Write the streaming data using foreachBatch to send weather data to Event Hub
query = streaming_df.writeStream.foreachBatch(process_batch).start()

query.awaitTermination()

# Close the producer after termination
producer.close()
